## Setup

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

C:\Users\doree\AppData\Local\Temp\ipykernel_3296\2359398814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data_train = pd.read_csv("../data/BikeRentalDaily_train.csv", sep=";", index_col=0).sort_index()

In [3]:
data_test = pd.read_csv("../data/prepared_test.csv", sep=";", index_col=0).sort_index()

## 1 Minimal Preprocessing

**Create a baseline linear regression model using the originally provided training 
dataset with minimal preprocessing and evaluate it with your validation dataset based on accuracies (MAE) and coefficient of determination (𝑅𝑅2)**

Während unserer ersten Analysen in Task 1 ist uns aufgefallen, dass die Variablen `season` und `hum` fehlende Werte aufweisen. Als Default-Methode und im Sinne von Minimal Preprocessing sollen die Zeilen mit fehlenden Werten entfernt werden.
Optionale Methoden zum Umgang mit fehlenden Werten werden im weiteren Verlauf addressiert.

In [4]:
data_train = data_train.dropna()
# data_test = data_test.dropna()

Desweiteren sollen die Zeilen mit den negativen Werten bei `windspeed` entfernt werden.

In [ ]:
negative_rows_windspeed = data[data['windspeed'] < 0].index

negative_rows_windspeed

In [ ]:
data_train = data_train.drop(negative_rows_windspeed)
# auch hier für test?

Auch die Wochentage mit -1 werden bereinigt.

In [ ]:
def get_wday_by_date(row):
    if row["weekday"] > 0:
        return row
    weekday_shift = { # week starts on Sunday in dataset
        6: 0,
        0: 1,
        1: 2,
        2: 3,
        3: 4,
        4: 5,
        5: 6
    }
    dateformat = "%d.%m.%Y"
    row["weekday"] = weekday_shift[time.strptime(row["dteday"], dateformat).tm_wday]
    return row
data_new = data_new.apply(get_wday_by_date, axis=1)

## 2 Baseline Linear Regression Model

In [5]:
y_label = 'cnt'

features = data_train.drop(columns=[y_label, 'dteday']).columns

X = data_train[features]

y = data_train[y_label]

Erneuter Split des Trainingsdatensets, um ein Validation Datset zu generieren

In [6]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state=42)

In [7]:
reg = LinearRegression()

In [ ]:
reg.fit(X, y)

In [ ]:
y_pred = reg.predict(X_validation)

In [ ]:
r2 = r2_score(y_validation, y_pred).round(2)
mae = mean_absolute_error(y_validation, y_pred).round(2)

print(f'R2 Score: {r2}')
print(f'Mean Absolute Error: {mae}')

## 3 Other Options for Missing Value Handling

**Preprocess the original datasets to address the identified data quality issues e.g. Missing values, Outliers, Features to be transformed (e.g. normalization). Check the effect of each preprocessing step with your validation data set* building additional linear regression models**

Einfluss auf R2 und MAE prüfen

Hier jetzt zum Beispiel verschiedene Imputation Strategien

Wie wir bereits in Task 1 erarbeitet haben, weisen die Spalten `season` und `hum` fehlende Werte auf. Im ersten Schritt haben wir diese Zeilen entfernt. Nun wollen wir die fehlenden Werte imputieren.

In [ ]:
msno.matrix(data)

Die fehlenden Werte in der Spalte `season` werden anhand des vorliegenden Datums in der entsprechenden Zeile aufgefüllt. 

`get_season_by_date` bestimmt die Jahreszeit anhand eines Datums im Format "dd.mm". Sie verwendet das Modul time, um feste Zeitpunkte für den Beginn der Jahreszeiten (Frühling, Sommer, Herbst, Winter) festzulegen und vergleicht dann das eingegebene Datum mit diesen Zeitpunkten. Die Funktion gibt eine Ganzzahl zurück, die die entsprechende Jahreszeit repräsentiert (1 für Frühling, 2 für Sommer, 3 für Herbst, 4 für Winter). Beachte jedoch, dass die Funktion Schwierigkeiten mit dem 29. Februar haben könnte, wenn keine Jahreszahl angegeben ist, da sie die Schaltjahre nicht berücksichtigt.
Es gibt vier fest definierte Zeitpunkte für den Beginn jeder Jahreszeit (Frühling, Sommer, Herbst, Winter). Diese Zeitpunkte sind auf den 20. März, 20. Juni, 20. September und 20. Dezember festgelegt.

In [ ]:
# datetime.datetime kann nicht mit 29. Feb umgehen, wenn keine Jahreszahl dabei ist
import time
def get_season_by_date(date: str):
    """Derive season by date"""
    dateformat = "%d.%m"

    # Season beginnings
    spring = time.strptime("20.03", dateformat)
    summer = time.strptime("20.06", dateformat)
    autumn = time.strptime("20.09", dateformat)
    winter = time.strptime("20.12", dateformat)

    date_p = time.strptime(date[:5], dateformat)

    if date_p < spring:
        return 1
    elif spring < date_p < summer:
        return 2
    elif summer < date_p < autumn:
        return 3
    elif autumn < date_p < winter:
        return 4
    else:
        return 1

In [ ]:
def season_wrapper(row):
    row["season"] = get_season_by_date(row["dteday"])
    return row
data_new = data.apply(season_wrapper, axis=1)
data_new

Fehlende Werte in der Spalte `hum` werden durch den Mittelwert ersetzt. (Wir können über andere Ersetzungswerte noch diskutieren!)

In [ ]:
data_new["hum"].fillna(np.mean(data_new["hum"]), inplace=True)

In [ ]:
msno.matrix(data_new)

## 4 Outlier Handling

Einfluss auf R2 und MAE prüfen

Es ist zu sehen, dass die Gesamtanzahl und die Zahl der nicht registrierten Nutzer einen sehr hohen Maximalwert haben. Die Gesamtzahl wird aus registrierten Nutzern + nicht registrierten Nutzern hergeleitet -> Die Outlier werden mittels der `casual` Spalte ermittelt.

In [ ]:
from scipy import stats
zscores = stats.zscore(data_new["casual"])
thresh = 3.0
outliers = data_new[abs(zscores) > thresh]
outliers

Anschließend erfolgt die Entfernung dieser Werte.

In [ ]:
data_no_out = data_new.drop(outliers.index, errors="ignore")

## 5 Feature Transformation

One Hot Encoding

In [ ]:
data_d = pd.concat([data_no_out, pd.get_dummies(data_no_out["weekday"], prefix="weekday", dtype=int)], axis=1)
data_d = pd.concat([data_d, pd.get_dummies(data_no_out["weathersit"], prefix="weathersit", dtype=int)], axis=1)
data_d.drop(columns=["weekday", "weathersit"], inplace=True)

## 6 Application on Test Dataset

**Finally, check your best model and final data preparation, applying them to the test dataset**

Nur das beste Model wird am Ende auf dem Test Dataset getestet!

## 7 Visualization of Evaluation Results

Export the pre-processed training, validation and test dataset to a CSV